In [1]:
# Standard framework
import pandas as pd, numpy as np

# We'll use bqplot for interactive plotting
import bqplot
from bqplot import pyplot as plt

# For our widgets:
import ipywidgets as widgets

In [2]:
# Import the data, the trained model and data preparation pipeline
# See the notebook 6.0-model_deployment-create_model 
# for details

In [3]:
from pycaret.datasets import get_data
from pycaret.regression import load_model, predict_model

In [4]:
df = get_data('bike', verbose=False)

In [5]:
# For convenience, we change the data types for the two boolean features:
df['holiday'] = df['holiday'].astype(bool)
df['workingday'] = df['workingday'].astype(bool)

In [6]:
# Download model. This takes a bit of time but only has to be done once 
# (per session)
import urllib
import os

if not os.path.isfile('saved_model_161121.pkl'):
    urllib.request.urlretrieve('https://www.dropbox.com/s/zrg2u479rt0ef6u/saved_model_161121.pkl?dl=1', filename='saved_model_161121.pkl')

In [7]:
model = load_model('saved_model_161121', verbose=False)

In [8]:
## Create some widgets for our webapp

In [9]:
## Widgets for entering feature values:

# Date feature

date_widget = widgets.DatePicker(
    value = pd.datetime(2011,1,1)
)

# Boolean features:

holiday_widget = widgets.Checkbox(
    value=False,
    description='Holiday?:',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

workingday_widget = widgets.Checkbox(
    value=True,
    description='Working day?:',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Numerical features

season_widget = widgets.Dropdown(
    options = [('Winter', 1), ('Spring',2), ('Summer',3), ('Fall',4)],
    value = 1,
    description = "Season",
    disabled = False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')    
    
)

year_widget = widgets.Dropdown(
    options = [('2011',0), ('2012', 1)],
    value = 0,
    description = "Year",
    disabled = False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')    
    
)

month_widget = widgets.Dropdown(
    options = range(1,13),
    value = 1,
    description='Month 1-12',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

hr_widget = widgets.Dropdown(
    options = range(24),
    value=0,
    description='Hour 0-23',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

weekday_widget = widgets.Dropdown(
    options = range(7),
    value = 6,
    description = "Weekday",
    disabled = False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')    
       
)

weather_widget = widgets.Dropdown(
    options = [('Clear, Few clouds, Partly cloudy, Partly cloudy', 1), 
               ('Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist', 2), 
               ('Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds', 3),
               ('Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog', 4)],
    value = 1,
    description = "Weather",
    disabled = False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')    
       
)

temp_widget = widgets.BoundedFloatText(
    value = 0.24,
    min=0.,
    max=1.,
    description = 'Normalized temperature in Celsius',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')    
)


atemp_widget = widgets.BoundedFloatText(
    value = 0.28,
    min=0.,
    max=1.,
    description = 'Normalized feeling temperature in Celsius',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')    
)

hum_widget = widgets.BoundedFloatText(
    value = 0.81,
    min = 0.,
    max = 1.,
    description = 'Normalized humidity',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')    
)

windspeed_widget = widgets.BoundedFloatText(
    value = 0.,
    min = 0.,
    max = 1.,
    description = 'Normalized windspeed',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='80%')    
)


all_widgets = [
    
    date_widget,
    holiday_widget,
    workingday_widget,
    season_widget,
    year_widget,
    month_widget,
    hr_widget,
    weekday_widget,
    weather_widget,
    temp_widget,
    atemp_widget,
    hum_widget,
    windspeed_widget    
]

In [10]:
# We also want to display some plots showing features of the training data:

feature = widgets.SelectMultiple(
    options = df.columns[2:], # Drop the instant value and the date
    value = ['cnt'],
    description='Feature',
    disabled=False,
    layout = widgets.Layout(width='50%', height='80px')
)

In [11]:
# A function to display the plots
def update_plots(feature):
    selected_values = list(feature)

    # If one boolean feature is selected, show a bar chart
    if (len(selected_values) < 2) and (len(df[selected_values].value_counts())==2):
        fig = plt.figure(title=selected_values[0])
        plt.bar(x=[str(i) for i in np.unique(df[selected_values])], y=df[selected_values].value_counts().values)
        plt.show()
    
    # If one numerical feature is selected, show a histogram
    elif (len(selected_values) < 2) and (df[selected_values[0]].dtype==np.int64):
        fig = plt.figure(title=selected_values[0])
        plt.hist(df[selected_values].values)
        plt.show()
        
    # If two numerical features are selected, show a scatter plot
    elif len(selected_values) == 2 and (df[selected_values[0]].dtype==float and df[selected_values[1]].dtype==float):
        df[selected_values].plot(x=selected_values[0], y=selected_values[1], 
                                 kind='scatter', title=f'{selected_values[0]} versus {selected_values[1]}',
                                figsize=(6,6))
        
    return None

In [12]:
# A function to output the selected feature values
from IPython.core.display import HTML
def selected_feature_values(dteday, season, yr, mnth, hr, holiday, weekday, 
                             workingday, weathersit, temp, atemp, hum, windspeed, instant=1):
    
    html = f""" <h4>Your Selections:</h4>
        <ul style="line-height: 1.5">
            <li>Date: {dteday}</li>
            <li>Season: {season}</li>
            <li>Year: {yr}</li>
            <li>Month: {mnth}</li>
            <li>Hour: {hr}</li>
            <li>Holiday status: {holiday}</li>
            <li>Weekday: {weekday}</li>
            <li>Working day status: {workingday}</li>
            <li>Weather situation: {weathersit}</li>
            <li>Temperature: {temp}</li>
            <li>Felt temperature: {atemp}</li>
            <li>Humidity: {hum}</li>
            <li>Windspeed: {windspeed}</li>            
        </ul>
        <BR>
        """
    display(HTML(html))

In [13]:
# A function to output the prediction for the selected feature values
from IPython.core.display import HTML
def get_pred(dteday, season, yr, mnth, hr, holiday, weekday, 
                             workingday, weathersit, temp, atemp, hum, windspeed, instant=1, cnt=99):
    
    
    pred_df = pd.DataFrame([{
        'dteday': dteday,
        'season': season, 
        'yr': yr, 
        'mnth': mnth, 
        'hr': hr, 
        'holiday':holiday, 
        'weekday': weekday,                    
        'workingday':workingday, 
        'weathersit':weathersit, 
        'temp':temp, 
        'atemp':atemp, 
        'hum':hum, 
        'windspeed':windspeed, 
        'instant':-1, 
    }])
    

    pred = predict_model(model, data=pred_df)
    
    html = f"""  
      <hr>
      The model predicts that there will be {round(pred.Label.values[0],2)} trips taken

    """
    display(HTML(html))

In [14]:
selected = widgets.interactive_output(
    selected_feature_values, 
    {
    'dteday': date_widget,
    'holiday':holiday_widget,
    'workingday':workingday_widget,
    'season':season_widget,
    'yr':year_widget,
    'mnth':month_widget,
    'hr':hr_widget,
    'weekday':weekday_widget,
    'weathersit':weather_widget,
    'temp':temp_widget,
    'atemp':atemp_widget,
    'hum':hum_widget,
    'windspeed':windspeed_widget 
    }
)

In [15]:
prediction = widgets.interactive_output(
    get_pred, 
    {
    'dteday': date_widget,
    'holiday':holiday_widget,
    'workingday':workingday_widget,
    'season':season_widget,
    'yr':year_widget,
    'mnth':month_widget,
    'hr':hr_widget,
    'weekday':weekday_widget,
    'weathersit':weather_widget,
    'temp':temp_widget,
    'atemp':atemp_widget,
    'hum':hum_widget,
    'windspeed':windspeed_widget 
    }
)

In [16]:
# Deploy the model

In [17]:
title_html = f"""
<h2>Bike-sharing demo app</h2>
Several features are used to predict the number of bikes used. 
"""

feature_values_html = f"""
<hr>
<h3> Enter the feature values here:</h3> 
"""

plots_html = f"""
<hr>
<h3> Investigate training data</h3>
You can investigate the values of the features used to train the model here: 
"""

pred_html = f"""
<hr>
<h3>Model prediction:</h3>
"""

In [18]:
appcont = [widgets.HTML(title_html, 
                        layout=widgets.Layout(margin='0 0 3em 0', max_width='1000px')),
          widgets.HTML(feature_values_html, 
                        layout=widgets.Layout(margin='0 0 3em 0', max_width='1000px'))]

In [19]:
app = widgets.VBox(appcont)

In [20]:
display(app)

In [21]:
for w in all_widgets:
    display(w)

DatePicker(value=datetime.datetime(2011, 1, 1, 0, 0))

Checkbox(value=False, description='Holiday?:', layout=Layout(width='80%'), style=DescriptionStyle(description_…

Checkbox(value=True, description='Working day?:', layout=Layout(width='80%'), style=DescriptionStyle(descripti…

Dropdown(description='Season', layout=Layout(width='80%'), options=(('Winter', 1), ('Spring', 2), ('Summer', 3…

Dropdown(description='Year', layout=Layout(width='80%'), options=(('2011', 0), ('2012', 1)), style=Description…

...

Dropdown(description='Weather', layout=Layout(width='80%'), options=(('Clear, Few clouds, Partly cloudy, Partl…

BoundedFloatText(value=0.24, description='Normalized temperature in Celsius', layout=Layout(width='80%'), max=…

BoundedFloatText(value=0.28, description='Normalized feeling temperature in Celsius', layout=Layout(width='80%…

BoundedFloatText(value=0.81, description='Normalized humidity', layout=Layout(width='80%'), max=1.0, style=Des…

BoundedFloatText(value=0.0, description='Normalized windspeed', layout=Layout(width='80%'), max=1.0, style=Des…

In [22]:
display(selected)

Output()

In [23]:
display(widgets.HTML(pred_html, 
                        layout=widgets.Layout(margin='0 0 3em 0', max_width='1000px')))

HTML(value='\n<hr>\n<h3>Model prediction:</h3>\n', layout=Layout(margin='0 0 3em 0', max_width='1000px'))

In [24]:
display(prediction)

Output()

In [25]:
display(widgets.HTML(plots_html, 
                        layout=widgets.Layout(margin='0 0 3em 0', max_width='1000px')))

HTML(value='\n<hr>\n<h3> Investigate training data</h3>\nYou can investigate the values of the features used t…

In [26]:
widgets.interactive(update_plots, feature=feature)

interactive(children=(SelectMultiple(description='Feature', index=(12,), layout=Layout(height='80px', width='5…